In [1]:
!pip install Pillow

In [2]:
import subprocess
import sys
from logging import exception

from fontTools.misc.psOperators import ps_string

# pip 설치 시도
subprocess.check_call([sys.executable, "-m", "ensurepip", "--default-pip"])
# Pillow 설치
subprocess.check_call([sys.executable, "-m", "pip", "install", "Pillow"])

0

In [3]:
!pip install pygame

In [4]:
import pygame
import os

pygame.mixer.init()

# 윈도우 기본 미디어 폴더 경로
win_media_path = "C:/Windows/Media/"

sound_correct = pygame.mixer.Sound(win_media_path + "Speech On.wav")
sound_wrong = pygame.mixer.Sound(win_media_path + "Speech Misrecognition.wav")

C:\Code\PythonProject\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.1,
    shear_range=0.2
)

model = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(56, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(datagen.flow(train_images, train_labels, batch_size=32),
          epochs=20)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
model.save('tensorflow_model.keras')
print('\n테스트 정확도:', test_acc)

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.8190 - loss: 0.5639
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9164 - loss: 0.2709
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9320 - loss: 0.2198
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9396 - loss: 0.1927
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9453 - loss: 0.1772
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9487 - loss: 0.1659
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9527 - loss: 0.1548
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9548 - loss: 0.1459
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9574 - loss: 0.1377
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9578 - loss: 0.1367
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9607 - loss: 0.1272
Epoch 12/20
1875/1

In [92]:
import cv2
import numpy as np
import time
import random
from PIL import ImageFont, ImageDraw, Image
import pygame
from tensorflow.keras.models import load_model

### 변수 선언 및 초기화 ###
game_mode = "WAITING"
game_level = 1 # 1 : 초급, 2 : 중급, 3 : 고급(고급은 구상만)
answer = 0 # 정답 숫자용 변수
quiz_text = '' # answer를 화면에 띄우기 위한 putText용
time_limit = 10 # 제한시간 10초
start_time = 0 # 시작시간
ready_time = 3

# 버튼 위치 정의 [x1, y1, x2, y2]
btn1_rect = [100, 350, 300, 430] # 초급 버튼
btn2_rect = [340, 350, 540, 430] # 중급 버튼

# 오답 메시지 관련 변수
wrong_msg = False # 오답 메시지 표시할지 말지
wrong_msg_time = 0 # 오답 메시지용 time변수
wrong_duration = 1.0 # 오답메세지 유지시간

# 퀴즈 변수
quiz_count = 0      # 현재 몇 번째 문제인지
max_quiz = 10       # 총 문제 수
total_score = 0.0   # 누적 점수

# 마우스 호버효과를 위한 현재 마우스 포지션
current_mouse_pos = [0, 0] # [x, y] 좌표 저장

# 정,오답 시 이펙트 -> 화면 번쩍임
flash_time = 0
flash_color = (0,0,0)

# floating score 애니메이션
score_effects = [] # [{'val': 8.5, 'start_time': t, 'pos': (x, y)}, ...]

pygame.mixer.init()
media_path = "C:/Windows/Media/"
sound_correct = pygame.mixer.Sound(media_path + "Speech On.wav")
sound_wrong = pygame.mixer.Sound(media_path + "Speech Misrecognition.wav")

try:
    model = load_model('tensorflow_model.keras')
    print("모델 로드 성공!")
except Exception as e:
    exit()

# 버튼 ui 적용하기
def draw_button(img, text, rect, base_color, text_color =(255,255,255)):
    x1, y1, x2, y2 = rect
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    try:
        _font = ImageFont.truetype("C:/Windows/Fonts/malgunbd.ttf", 30)
    except:
        _font = ImageFont.load_default()

    # 1. 외곽선이 있는 둥근 사각형 (버튼 몸체)
    draw.rounded_rectangle([x1+3, y1+3, x2+3, y2+3], radius=15, fill=(30, 30, 30)) # 그림자
    draw.rounded_rectangle([x1, y1, x2, y2], radius=15, fill=base_color, outline=(255, 255, 255), width=2)

    # 2. 텍스트 중앙 정렬 계산
    w, h = x2 - x1, y2 - y1
    tw, th = draw.textsize(text, font=_font) if hasattr(draw, 'textsize') else (70, 30)
    text_pos = (x1 + (w - tw) / 2, y1 + (h - th) / 2 - 5)

    # 3. 버튼 텍스트 그리기
    draw.text(text_pos, text, font=_font, fill=(255, 255, 255)) # 버튼 글씨는 흰색 고정

    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# 한글 출력을 위해서는 opencv2 프레임을 Pillow로 전환 후, 한글로 그린 뒤에 다시 opencv2에 돌려주는 함수가 필요하다.
def print_korean(img, text, pos, f_size, _color):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    try:
        _font = ImageFont.truetype("C:/Windows/Fonts/malgun.ttf", f_size)
    except Exception as e:
        _font = ImageFont.load_default()
        print(f"error : {e}")

    # 글씨 외곽선 그리기
    x, y = pos
    stroke_color = (0, 0, 0) # 검은색 외곽선
    for adj in range(0, 2): # 두께 조절 (2픽셀)
        draw.text((x+adj, y), text, font=_font, fill=stroke_color)
        draw.text((x, y+adj), text, font=_font, fill=stroke_color)

    draw.text(pos,text, font=_font, fill=_color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGRA)

### 난이도 선택을 위한 마우스 콜백
def mouse_callback(event, _x, _y, _flags, _param):
    global game_mode, game_level, start_time,current_mouse_pos
    # 마우스가 움직일 때마다 좌표 업데이트
    if event == cv2.EVENT_MOUSEMOVE:
        current_mouse_pos = [_x, _y]
    if event == cv2.EVENT_LBUTTONDOWN and game_mode == "WAITING":
        if (btn1_rect[0] < _x < btn1_rect[2]) and (btn1_rect[1] < _y < btn1_rect[3]):
            game_level = 1
            game_init()
            game_mode = "READY"
            print("초급")
        elif (btn2_rect[0] < _x < btn2_rect[2]) and (btn2_rect[1] < _y < btn2_rect[3]):
            game_level = 2
            game_init()
            game_mode = "READY"
            print("중급")
        else: # 난이도 고급
            pass

### 퀴즈 메소드
def quiz_fuc(lv):
    global answer, quiz_text
    if lv == 1: # 초급
        answer = random.randint(1,9) # 1~9까지 숫자 랜덤
        quiz_text = "숫자 : " + str(answer)
        print(quiz_text)
        pass
    elif lv == 2: # 중급
        op = random.choice(['+', '-'])
        if op == '+':
            a = random.randint(1, 8)
            b = random.randint(1, 9-a) # 합이 9가 넘지 않도록, a 기반으로 랜덤
            answer = a + b
        else:
            a = random.randint(2, 9)
            b = random.randint(1, a-1) # 합이 음수가 되지않올고 a보다 작은 숫자만 나오도록
            answer = a - b
        quiz_text = f"QUIZ : {a} {op} {b} = ?" # ex) 5 + 4 = ? / 9 - 4 = ?
        print(quiz_text)
    elif lv == 3: # 고급
        print("여긴 아이디어의 단계입니다.")
    else:
        pass

def game_init():
    global quiz_count, total_score, start_time
    quiz_count = 1
    total_score = 0.0
    quiz_fuc(game_level)
    start_time = time.time()


window_name = "Tensorflow Game"
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, mouse_callback)
###
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("웹캠을 열 수 없습니다.")
    exit()

###
while True:
    ret, frame = cap.read() # 리턴값과 프레임
    if not ret: # false면
        print("프레임을 가져 올 수 없습니다.")
        break
    key = cv2.waitKey(1) & 0xFF

    flip_frame = cv2.flip(frame, 1) # y축을 기준으로 뒤집어 주세요 -> 거울 반전
    height, width, color = frame.shape # 높이, 폭, color인데 color는 딱히 안쓸거임
    center_x, center_y = width // 2, height // 2 # x와 y의 중심점

    offset_y = 60 # 화면 위아래 조절하려고 만든 변수
    roi = flip_frame[center_y - 150 + offset_y : center_y + 150 + offset_y, center_x - 150:center_x + 150]    # roi 영역 만들기
    # #x,y중심점을 기준으로 150씩의 거리를 가진 영역 만들기 -> 300x300

    font = cv2.FONT_HERSHEY_SIMPLEX

    # game_mode 구현.
    if game_mode == "WAITING": # 난이도 선택
        flip_frame = print_korean(flip_frame, "숫자 맞추기 게임", (center_x-160, center_y-200), 40, (0,0,0))
        flip_frame = print_korean(flip_frame, "난이도를 정하세요 [ 초급, 중급 ]", (center_x-220, center_y-130), 30, (255,255,0))

        mx, my = current_mouse_pos # 현재 마우스 좌표

        # 초급 버튼 호버효과 - 더 밝은 보라색
        if (btn1_rect[0] < mx < btn1_rect[2]) and (btn1_rect[1] < my < btn1_rect[3]):
            flip_frame = draw_button(flip_frame, "초 급", btn1_rect, (200, 100, 255))
        else:
            flip_frame = draw_button(flip_frame, "초 급", btn1_rect, (150, 50, 200))
        # 중급 버튼 호버효과 - 더 밝은 청록색
        if (btn2_rect[0] < mx < btn2_rect[2]) and (btn2_rect[1] < my < btn2_rect[3]):
            flip_frame = draw_button(flip_frame, "중 급", btn2_rect, (100, 220, 220))
        else:
            flip_frame = draw_button(flip_frame, "중 급", btn2_rect, (50, 150, 150))
    elif game_mode == "READY":
        flip_frame = print_korean(flip_frame, "게임이 곧 시작됩니다!!!", (center_x-215, center_y-120), 40, (255,0,0))

        delay_time = time.time() - start_time
        count = int(ready_time - delay_time) + 1
        if count > 0:
            flip_frame = print_korean(flip_frame, str(count), (center_x - 15, center_y - 25), 50, (255,0,0))
        else:
            start_time = time.time()
            game_mode = "PLAYING"


    elif game_mode == "PLAYING":
        cv2.rectangle(flip_frame, (center_x - 150, center_y - 150 + offset_y),
                  (center_x + 150, center_y + 150 + offset_y), (0,0,255), 2) # 외곽선 그리기

        delay_time = time.time() - start_time
        remain_time = max(0, time_limit - delay_time)

        # 상단 정보 표시
        flip_frame = print_korean(flip_frame, quiz_text, (0, 60), 40, (255, 255, 0))
        flip_frame = print_korean(flip_frame, f"남은 시간: {remain_time:.1f}초", (0, 110), 30, (0, 0, 255))
        # 2. 진행 상황 표시 (중상단)
        flip_frame = print_korean(flip_frame, f"{quiz_count} / {max_quiz}", (center_x - 50, 20), 35, (255, 255, 255))
        # 3. 실시간 점수 표시 (우상단)
        flip_frame = print_korean(flip_frame, f"SCORE: {total_score:.1f}", (width - 220, 20), 35, (0, 255, 0))

        if wrong_msg: # 오답메세지를 출력했는지 확인
            if time.time() - wrong_msg_time < wrong_duration:
                flip_frame = print_korean(flip_frame, "오답!", (width-100, 100), 30, (255,0,0))
        else:
            wrong_time = False

        if key == ord('c') or key == ord('C'):
            correct_roi = cv2.flip(roi, 1)
            gray_image = cv2.cvtColor(correct_roi, cv2.COLOR_BGR2GRAY)
            gauss_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
            _, otsu_thread = cv2.threshold(gauss_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(otsu_thread, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if contours:
                cnt = max(contours, key=cv2.contourArea)
                x, y, w, h = cv2.boundingRect(cnt)
                digit = otsu_thread[y:y+h, x:x+w]

                kernel = np.ones((3, 3), np.uint8)
                digit = cv2.dilate(digit, kernel, iterations=1)

                target_size = 18
                if w > h:
                    new_w = target_size
                    new_h = int(h * (target_size / w))
                else:
                    new_h = target_size
                    new_w = int(w * (target_size / h))

                digit_resized = cv2.resize(digit, (new_w, new_h), interpolation=cv2.INTER_AREA)

                final_img = np.zeros((28, 28), dtype=np.uint8)
                start_x = (28 - new_w) // 2
                start_y = (28 - new_h) // 2
                final_img[start_y:start_y+new_h, start_x:start_x+new_w] = digit_resized

                final_img = cv2.GaussianBlur(final_img, (3, 3), 0)
                resized_img = final_img
            else:
                resized_img = np.zeros((28, 28), dtype=np.uint8)

            cv2.imshow('What AI Sees', resized_img)
            test_data = resized_img.reshape(1, 28, 28, 1).astype('float32') / 255.0

            try:
                pred_img = model.predict(test_data , verbose=0)
                result = np.argmax(pred_img)
                print(f"인식 결과 : {result}, 정답 : {answer}")

                # print("Raw Sofxmax:", pred_img[0])
                # print("Prediced Class:", np.argmax(pred_img[0]))
                # print("Confidence:", np.max(pred_img[0]))

                if result == answer:
                   print("정답!")
                   wrong_msg = False
                   sound_correct.play()

                   total_score += remain_time
                   plus_score = remain_time

                   # floating score
                   score_effects.append({
                       'val': plus_score,
                       'start_time': time.time(),
                       'x': width - 150, # SCORE 표시 근처
                       'y': 60
                   })
                   # 화면 반짝임
                   flash_color = (0, 255, 0) # 초록색
                   flash_time = time.time()

                   if quiz_count >= max_quiz:
                       game_mode = "END"
                   else:
                       quiz_count += 1
                       quiz_fuc(game_level) # 다음 문제 생성
                       start_time = time.time() # 시간 초기화
                else:
                   print("오답!")
                   wrong_msg = True
                   sound_wrong.play()

                   flash_color = (0, 0, 255) # 빨간색
                   flash_time = time.time()

                   wrong_msg_time = time.time() #c연타해도 마지막 기준 1초 동안 메세지 출력하도록
            except Exception as e:
                print(f"error : {e}")
        if remain_time <= 0 :
            if quiz_count >= max_quiz:
                game_mode = "END"
            else:
                quiz_count += 1
                quiz_fuc(game_level)
                start_time = time.time()

    elif game_mode == "END":
        level_str = "초급" if game_level == 1 else "중급"
        flip_frame = print_korean(flip_frame, "GAME OVER", (center_x - 120, center_y - 120), 50, (0, 0, 255))
        flip_frame = print_korean(flip_frame, f"난이도 : {level_str}", (center_x - 100, center_y - 40), 30, (255, 255, 255))
        flip_frame = print_korean(flip_frame, f"최종 점수 : {total_score:.1f}점", (center_x - 120, center_y + 10), 35, (0, 255, 255))
        flip_frame = print_korean(flip_frame, "Enter를 눌러 메인으로", (center_x - 140, center_y + 80), 25, (200, 200, 200))

        if key == 13: # 엔터
            game_mode = "WAITING"

    # Floating Score 애니메이션
    curr_t = time.time()
    for effect in score_effects[:]:
        elapsed = curr_t - effect['start_time']
        if elapsed > 1.0: # 1초 지나면 사라짐
            score_effects.remove(effect)
            continue

        # y값은 위로(-), 투명도는 낮게 계산
        move_up = int(elapsed * 60)
        # print_korean 함수를 써서 출력
        flip_frame = print_korean(flip_frame,
                                  f"+{effect['val']:.1f}", (effect['x'], effect['y'] - move_up), 30, (0, 255, 255))


    # 화면 번쩍임을 화면에 그리기
    if time.time() - flash_time < 0.2: # 0.2초 동안만 번쩍
        overlay = flip_frame.copy()
        cv2.rectangle(overlay, (0, 0), (width, height), flash_color, -1)
        # addWeighted로 투명도 조절 (0.3은 30% 농도)
        flip_frame = cv2.addWeighted(overlay, 0.3, flip_frame, 0.7, 0)

    cv2.imshow(window_name, flip_frame)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


모델 로드 성공!
숫자 : 4
초급
인식 결과 : 4, 정답 : 4
정답!
숫자 : 8
인식 결과 : 8, 정답 : 8
정답!
숫자 : 1
인식 결과 : 1, 정답 : 1
정답!
숫자 : 3
인식 결과 : 3, 정답 : 3
정답!
숫자 : 3
인식 결과 : 3, 정답 : 3
정답!
숫자 : 3
인식 결과 : 3, 정답 : 3
정답!
숫자 : 7
인식 결과 : 7, 정답 : 7
정답!
숫자 : 3
인식 결과 : 3, 정답 : 3
정답!
숫자 : 5
인식 결과 : 8, 정답 : 5
오답!
인식 결과 : 4, 정답 : 5
오답!
인식 결과 : 5, 정답 : 5
정답!
숫자 : 4
인식 결과 : 4, 정답 : 4
정답!


In [78]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 로드 및 4차원 변형
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1) / 255.0
test_images = test_images.reshape(-1, 28, 28, 1) / 255.0

# 증강 설정 강화 (실제 웹캠 환경에 맞춤)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

# CNN 모델 구조 (인식률 대폭 향상)
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    keras.layers.BatchNormalization(), # 추가: 데이터 정규화
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    keras.layers.BatchNormalization(), # 추가
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3), # Dropout 상향
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 증강 데이터로 더 많이 학습 (CNN은 데이터가 많이 필요함)
model.fit(datagen.flow(train_images, train_labels, batch_size=64), epochs=30)

model.save('tensorflow_model_cnn.keras')

Epoch 1/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.8165 - loss: 0.5768
Epoch 2/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9331 - loss: 0.2208
Epoch 3/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9480 - loss: 0.1754
Epoch 4/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9564 - loss: 0.1501
Epoch 5/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9592 - loss: 0.1356
Epoch 6/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9641 - loss: 0.1233
Epoch 7/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9663 - loss: 0.1173
Epoch 8/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9676 - loss: 0.1116
Epoch 9/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9694 - loss: 0.1057
Epoch 10/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9703 - loss: 0.1015
Epoch 11/30
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9717 - loss: 0.0962
Epoch 12/30
938/938 ━━━━━━━━━━